In [1]:
from src import Frame, Rectangle, VideoReader, VideoWriter, find_module
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

class Trackbar:
    def __init__(self, name, window, value, count, on_change):
        self.name = name
        self.window = window
        self._range = (0,count)
        cv.createTrackbar(self.name, self.window, value, count, on_change)

    @property
    def pos(self):
        return cv.getTrackbarPos(self.name, self.window)
    
    @property
    def min(self): return self._range[0]
    
    @property
    def max(self): return self._range[1]
    
    @pos.setter
    def pos(self, pos):
        cv.setTrackbarPos(self.name, self.window, pos)
        
    @min.setter
    def min(self, val):
        self._range = (val, self._range[1])
        cv.setTrackbarMin(self.name, self.window, val)
    
    @max.setter
    def max(self, val):
        self._range = (self._range[0], val)
        cv.setTrackbarMax(self.name, self.window, val)

        
def play(filename):
    vid = VideoReader(filename)
    cv.namedWindow(filename)
    cv.imshow(filename, vid[0])
    playing = True
    while True:
        key = cv.waitKey(0)
        if key not in [-1,0,1,2,3,32]:
            print(key)
        if key == 3 and vid.pos < vid.length - 1:
            cv.imshow(filename, vid[vid.pos])
        if key == 2 and vid.pos > 0:
            vid.pos -= 2
            cv.imshow(filename, vid[vid.pos])
        if key == 27 or key == 113:
            break
    vid.release()
    cv.destroyWindow(filename)


play('./tests/videos/modules.mp4')

0
1
1
0
1
0
1
0
0
1
0
0
0
0
1
1
1
27


In [ ]:
with VideoReader('./tests/videos/modules.mp4') as vid:
    with Window('hello', fps=30) as win:
        trackbar = Trackbar('meep', win, 100, 255)
        for frame in vid:
            win.show(frame)

In [ ]:
def on_button(*args, **kwargs):
    print('button', args, kwargs)

In [ ]:
class Window:
    def __init__(self, name, fps=None, size='auto', ratio='keep', gui='expanded', opengl=False):
        self.flags = 0
        assert size in ['normal','auto']
        assert ratio in ['free','keep']
        assert gui in ['normal','expanded']
        
        if size == 'auto':
            self.flags |= cv.WINDOW_AUTOSIZE
        if ratio == 'keep':
            self.flags |= cv.WINDOW_KEEPRATIO
        if gui == 'expanded':
            self.flags |= cv.WINDOW_GUI_EXPANDED
        if opengl:
            self.flags |= cv.WINDOW_OPENGL
        
        self.name = name
        self._title = name
        self.fps = fps
    
    @property
    def title(self):
        return self._title
    
    @title.setter
    def title(self, title):
        cv.setWindowTitle(self.name, title)
    
    def on_mouse(self, event, x, y, flags, userdata=None):
        print('mouse', event, x, y, flags, userdata)
    
    def on_draw(self, userdata=None):
        print('draw', args, kwargs)
    
    def create_trackbar(self):
        pass
    
    def get_mouse_wheel(self, flags):
        pass
    
    def show(self, img):
        cv.imshow(self.name, img)
        
        if self.fps is not None:
            key = self.waitKey(int(1000/self.fps))
            print(key)
    
    def waitKey(self, delay=0):
        return cv.waitKey(delay)
    
    @property
    def fullscreen(self): return bool(cv.getWindowProperty(self.name,cv.WIND_PROP_FULLSCREEN))
    
    @property
    def autosize(self): return bool(cv.getWindowProperty(self.name,cv.WND_PROP_AUTOSIZE))

    @property
    def free_ratio(self): return bool(cv.getWindowProperty(self.name,cv.WND_PROP_ASPECT_RATIO))
    
    @property
    def opengl(self): return cv.getWindowProperty(self.name,cv.WND_PROP_OPENGL)
    
    @property
    def visible(self): return cv.getWindowProperty(self.name, cv.WND_PROP_VISIBLE)
    
    @property
    def topmost(self): return cv.getWindowProperty(self.name, cv.WND_PROP_TOPMOST)
    
    @property
    def vsync(self): return cv.getWindowProperty(self.name, cv.WND_PROP_VSYNC)
    
    def __enter__(self):
        cv.namedWindow(self.name, self.flags)
        return self
    
    def __exit__(self,*args):
        cv.destroyWindow(self.name)


class VideoPlayer:
    def __init__(self, filename, fn):
        self.vid = VideoReader(filename)
        self.fps = self.vid.fps
        self.name = filename
        self.fn = fn
        self.playing = True
        cv.namedWindow(self.name)
        self.trackbar = Trackbar('frame', self.name, 0, self.vid.length, self.on_frame_change)
        self.open = True
        while self.open:
            if self.playing and self.vid.pos < self.vid.length - 1:
                self.on_frame_change(self.vid.pos + 1)
            self.wait_for_key()
    
    def wait_for_key(self):
#         key = cv.waitKey(0)
        key = cv.waitKey(int(1000/self.fps))
        if key != -1:
            print('Key pressed',key)
        if key == -1:
            pass
        elif key == 32: # Space
            self.playing = !self.playing
#         elif key == 3: # -> arrow
#             self.trackbar.pos += 1
#         elif key == 2: # <- arrow
#             self.trackbar.pos -= 1
        else:
            print(key)            
        
    def on_frame_change(self, value):
        frame = self.vid[value]
        cv.imshow(self.name, frame)
    
    def close(self):
        self.vid.release()
        cv.destroyWindow(self.name)


def mutations(frame):
    pass